In [1]:
import pandas as pd
import numpy as np
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

### Settings

In [2]:
# Directories
out_dir = os.path.join(os.pardir,"output")
src = os.path.join(os.pardir, "src")

# Raw data
middle_income_file = os.path.join(src, "HE0110K1.xlsx") # Middle income
median_income_file = os.path.join(src, "HE0110K2.xlsx") # Median income

### Helpers

In [3]:
def checkDir(directory):
    if not os.path.exists(directory):
        print directory + " did not exist. Creating it..."
        os.makedirs(directory)
    else:
        print directory + " already exists. No further action."
    return

### Extract entities - counties

In [4]:
def extract_entities_counties(data):
    entities_file = os.path.join(out_dir, "ddf--entities--county.csv")
    
    county = data[["Unnamed: 0", "Unnamed: 1"]].copy() # Copy entity columns
    county.rename(columns = {"Unnamed: 0": "county", "Unnamed: 1": "name"}, inplace=True) # Rename columns
    county["county"] = county["county"].astype(unicode).map(to_concept_id) # Make id string alphanumeric
    county["is--county"] = True
    
    return county

### Extract concepts

In [5]:
def extract_concepts(out_dir):
    concept_file = os.path.join(out_dir, "ddf--concepts.csv")
    
    concepts = ["Name", "Middle income", "Median income", "Year", "County"]
    df_con = pd.DataFrame([], columns = ["concept", "name", "concept_type"])

    df_con["name"] = concepts
    df_con["concept"] = df_con["name"].astype(unicode).map(to_concept_id)

    df_con["concept_type"] = "measure"
    df_con["concept_type"].iloc[0] = "string"
    df_con["concept_type"].iloc[3] = "time"
    df_con["concept_type"].iloc[4] = "entity_domain"
    
    # If not exist, create new
    # Else, add to existing file
    if os.path.isfile(concept_file):
        print concept_file + " exists. Adding new entries."
        data = pd.read_csv(concept_file, encoding="utf-8")
        df_con = pd.concat([data, df_con])
        df_con = df_con.drop_duplicates(subset=["concept"])
        
    return df_con

### Extract datapoints

In [6]:
def extract_datapoints(data, measure):
    dps = data.copy()
    dps.rename(columns = {"Unnamed: 0": "county", "Unnamed: 1": "name"}, inplace=True)

    income = dps[dps.columns[2:]]
    entities = dps[["county", "name"]]
    
    datapoints = pd.DataFrame([], columns=["county", "name", "year", measure])
    for year in income.columns:
        tmp = entities.copy()
        tmp["year"] = year
        tmp[measure] = income[year]*1000
        datapoints = pd.concat([datapoints,tmp])
        
    datapoints["county"] = datapoints["county"].astype(unicode).map(to_concept_id)
    
    return datapoints.sort_values(by=["county","year"])

### Main script

In [7]:
if __name__ == "__main__":
    
    # Load raw data
    files = [middle_income_file, median_income_file]
    measure = ["middle_income", "median_income"]
    first = True
    
    #Check if output dir exists, otherwise create it
    checkDir(out_dir)
    
    for i, infile in enumerate(files):
        print "Reading " + infile
        data = pd.read_excel(infile, skiprows=[0,1], parse_cols="A,B,G:AD", skip_footer=31, \
                                 converters={'Unnamed: 0': lambda x: str(x)}) 
        
        data.replace("..",0,inplace=True)
    
        if (first):
            # Extract entities (counties)
            county = extract_entities_counties(data)
            path = os.path.join(out_dir, "ddf--entities--county.csv")
            print "Printing " + path
            county.to_csv(path, index=False, encoding="utf-8")

            # Extract concepts
            concepts = extract_concepts(out_dir)
            path = os.path.join(out_dir, "ddf--concepts.csv")
            print "Printing " + path
            concepts.to_csv(path, index=False, encoding="utf-8")

            first = False
    
        # Extract datapoints
        datapoints = extract_datapoints(data, measure[i])
        path = os.path.join(out_dir, "ddf--datapoints--{}--by--county--year.csv".format(measure[i]))
        print "Printing " + path
        datapoints.to_csv(path, index=False, encoding="utf-8")
        
    # Create index file
    print("Creating index files...")
    create_index_file(out_dir)
    
    del county, concepts, data, datapoints    

../output already exists. No further action.
Reading ../src/HE0110K1.xlsx
Printing ../output/ddf--entities--county.csv
../output/ddf--concepts.csv exists. Adding new entries.
Printing ../output/ddf--concepts.csv
Printing ../output/ddf--datapoints--middle_income--by--county--year.csv
Reading ../src/HE0110K2.xlsx
Printing ../output/ddf--datapoints--median_income--by--county--year.csv
Creating index files...
